### 【問題1】クロスバリデーション
```
事前学習期間は検証データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。
具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。
```

In [4]:
#
# データ前準備
#

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_info_columns', 150)
pd.set_option('display.max_info_rows', 150)

df_base = pd.read_csv("../data/home-credit-default-risk/application_train.csv")

In [5]:
# 最初の値
x_names = [
    "DAYS_EMPLOYED", "REGION_RATING_CLIENT_W_CITY", "FLAG_DOCUMENT_3", "REGION_RATING_CLIENT", "DAYS_BIRTH",
    "AMT_REQ_CREDIT_BUREAU_YEAR", "OBS_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", "OWN_CAR_AGE", "DAYS_ID_PUBLISH",
    "DEF_30_CNT_SOCIAL_CIRCLE", "FLAG_DOCUMENT_7", "DEF_60_CNT_SOCIAL_CIRCLE", "DAYS_LAST_PHONE_CHANGE",
]
y_name = "TARGET"
df = df_base.loc[:, np.append(x_names, y_name)]
# null処理 
df = df.dropna().reset_index(drop=True)
x = df.loc[:, x_names]
y = df[y_name]

#
# クロスバリデーション
#
import lightgbm as lgbm
lgb = lgbm.LGBMRegressor()

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=4, random_state=0, shuffle=True)
result = cross_val_score(lgb, x, y, cv = kfold, scoring="roc_auc")
result.mean()

0.6267089973920692

In [14]:
x_names = [
    "DAYS_EMPLOYED", "REGION_RATING_CLIENT_W_CITY", "FLAG_DOCUMENT_3", "REGION_RATING_CLIENT", "DAYS_BIRTH",
    "AMT_REQ_CREDIT_BUREAU_YEAR", "OBS_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", "OWN_CAR_AGE", "DAYS_ID_PUBLISH",
    "DEF_30_CNT_SOCIAL_CIRCLE", "FLAG_DOCUMENT_7", "DEF_60_CNT_SOCIAL_CIRCLE", "DAYS_LAST_PHONE_CHANGE",
    "EXT_SOURCE_3"
]
# "FLAG_DOCUMENT_3" これが相関あるらしい

y_name = "TARGET"
df = df_base.loc[:, np.append(x_names, y_name)]
# null処理 
df = df.dropna().reset_index(drop=True)
x = df.loc[:, x_names]
y = df[y_name]

#
# クロスバリデーション
#
import lightgbm as lgbm
lgb = lgbm.LGBMRegressor()

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=4, random_state=0, shuffle=True)
result = cross_val_score(lgb, x, y, cv = kfold, scoring="roc_auc")
result.mean()

0.7046872986572721

### 【問題2】グリッドサーチ
```
これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。
グリッドサーチをパイプラインの中に組み込みましょう。
```

In [19]:
from sklearn.model_selection import GridSearchCV

params = {
    'num_leaves': [3, 5, 7, 9, 11],
    'min_child_weight': [0, 5, 15, 100, 300],
    'learning_rate': [0.1, 0.15, 0.2],
    'subsample': [0.01, 0.05, 0.1, 0.15],
}

grid_search = GridSearchCV(lgb, param_grid=params, cv=3, scoring="roc_auc")
grid_search.fit(x, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

0.7101901325016641
{'learning_rate': 0.15, 'min_child_weight': 0, 'num_leaves': 3}


### 【問題3】Kernelからの調査
```
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。
```

```
複数の特徴量を試しまして、EXT_SOURCE_3でapplicationを割ったのは効果がありました
application以外のcsvから新しい特徴量を作る
Previous_application.csvの最後の 3, 5番目の特徴量と最初の2, 4番目の特徴量のapplicationsについてそれぞれ集計は一番良いスコアになりました。
creditannuityratio: AMTCREDIT / AMTANNUITYの割合
credit_goods_price_ratio：AMT_CREDIT / AMT_GOODS_PRICEの割合
credit_downpayment：AMT_GOOD_PRICE – AMT_CREDITの差異
prev_PRODUCT_COMBINATION：一番直近の応募の特徴量

参考: https://data-analysis-stats.jp/kaggle/kaggle1%e4%bd%8d%e3%81%ae%e8%a7%a3%e6%9e%90%e6%89%8b%e6%b3%95%e3%80%80%e3%80%8chome-credit-default-risk-%e5%82%b5%e5%8b%99%e4%b8%8d%e5%b1%a5%e8%a1%8c%e3%81%ae%e4%ba%88%e6%b8%ac%e3%80%8d%e2%91%a1/
```

### 【問題4】高い汎化性能のモデル
```
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。
今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。
```

In [20]:
x_names = [
    "DAYS_EMPLOYED", "REGION_RATING_CLIENT_W_CITY", "FLAG_DOCUMENT_3", "REGION_RATING_CLIENT", "DAYS_BIRTH",
    "AMT_REQ_CREDIT_BUREAU_YEAR", "OBS_30_CNT_SOCIAL_CIRCLE", "OBS_60_CNT_SOCIAL_CIRCLE", "OWN_CAR_AGE", "DAYS_ID_PUBLISH",
    "DEF_30_CNT_SOCIAL_CIRCLE", "FLAG_DOCUMENT_7", "DEF_60_CNT_SOCIAL_CIRCLE", "DAYS_LAST_PHONE_CHANGE",
    "EXT_SOURCE_3"
]
y_name = "TARGET"
df = df_base.loc[:, np.append(x_names, y_name)]
# null処理 
df = df.dropna().reset_index(drop=True)
x = df.loc[:, x_names]
y = df[y_name]

#
# クロスバリデーション
#
import lightgbm as lgbm
lgb = lgbm.LGBMRegressor()

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=4, random_state=0, shuffle=True)
result = cross_val_score(lgb, x, y, cv = kfold, scoring="roc_auc")
result.mean()

0.7046872986572721